<a href="https://colab.research.google.com/github/ohohtani/fake_voice_detection/blob/main/FV_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import librosa
import torch                                        # 파이토치의 기본 모듈
import torch.nn as nn                               # 신경망
import torch.nn.functional as F                     # 신경망에서 자주 사용하는 함수들 ex) 활성화 함수
import torch.optim as optim                         # 최적화 알고리즘
from torch.utils.data import DataLoader, Dataset    # 데이터 로딩 및 처리
from torchvision import transforms                  # 데이터 변환을 위한

In [4]:
# 데이터셋 클래스 정의
class AudioDataset(Dataset):
    def __init__(self, dataset_path, labels, sample_rate, duration, n_mfcc, max_time_steps, transform=None):
        self.dataset_path = dataset_path
        self.labels = labels
        self.sample_rate = sample_rate
        self.duration = duration
        self.n_mfcc = n_mfcc
        self.max_time_steps = max_time_steps
        self.transform = transform
        self.file_list = list(labels.keys())

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        file_path = os.path.join(self.dataset_path, file_name + ".flac")
        label = self.labels[file_name]

        # 오디오 파일을 로드하고 전처리
        audio, _ = librosa.load(file_path, sr=self.sample_rate, duration=self.duration)
        mfcc = librosa.feature.mfcc(y=audio, sr=self.sample_rate, n_mfcc=self.n_mfcc)

        # 모든 MFCC가 동일한 길이(타임스텝)를 가지도록 설정
        if mfcc.shape[1] < self.max_time_steps:
            mfcc = np.pad(mfcc, ((0, 0), (0, self.max_time_steps - mfcc.shape[1])), mode='constant')
        else:
            mfcc = mfcc[:, :self.max_time_steps]

        # 변환이 정의되어 있으면 변환 적용
        if self.transform:
            mfcc = self.transform(mfcc)

        # 채널 추가 (1, N_MFCC, 타임스텝)
        mfcc = np.expand_dims(mfcc, axis=0)

        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


In [5]:
# 경로와 파라미터 정의
DATASET_PATH = "경로 지정"                          # 데이터셋이 저장된 디렉토리 경로
LABEL_FILE_PATH = "라벨 파일 경로"                  # 아래 클래스를 나타낼 파일임 (txt)
NUM_CLASSES = 2                                     # 클래스 수 (찐이랑 짭)
SAMPLE_RATE = 16000                                 # 오디오 파일의 샘플링 레이트 , 16000이 좀 기본적으로 많이 사용하는 느낌 , 이 값을 높임으로써 더 나은 음질을 얻을 수 있으나 계산이 복잡해짐.
DURATION = 5                                        # 오디오 클립의 길이 (초 단위) , 5초면 적절함. 다른 분들거 찾아봐도 길이 길어야 6초였음. (3~8초 정도로 바꿔가면서 테스트)
N_MFCC = 13                                         # MFCC 계수의 수 (일반적으로 13을 사용)
max_time_steps = 156                                # 파동 분석을 위해 신호를 작은 시간 단위로 나눔
                                                    # 프레임의 수 = (80000 - 2048) / 512 + 1 ≈ 155.56  (80000은 16000샘플링 * 5초, 2048과 512는 프레임 크기와 홉 기본 값)

In [6]:
labels = {}                                         # 파일 이름과 라벨을 저장할 딕셔너리

with open(LABEL_FILE_PATH, 'r') as label_file:      # 라벨 파일을 열고 각 라인의 내용을 읽음
    lines = label_file.readlines()

for line in lines:                                  # 각 라인을 처리하여 파일 이름과 라벨을 추출
    parts = line.strip().split()                    # 라인을 공백으로 나눔
    file_name = parts[1]                            # 파일 이름을 parts[1]에 저장함 parts[0]에는 다른 중요한 정보를 저장하거나 가독성을 위함, 여기서는 Speaker ID로 사용
    label = 1 if parts[-1] == "real" else 0         # 라벨이 찐이면 1, 아니면 0 (real or fake의 정보는 parts의 마지막 요소에 저장)
    labels[file_name] = label                       # 파일 이름과 라벨을 딕셔너리에 저장

FileNotFoundError: [Errno 2] No such file or directory: '라벨 파일 경로'

In [7]:
# 데이터셋 생
dataset = AudioDataset(DATASET_PATH, labels, SAMPLE_RATE, DURATION, N_MFCC, max_time_steps)

# 데이터셋을 학습 및 검증 세트로 분할
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# DataLoader를 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [11]:
# CNN 모델 클래스 정의
class CNNModel(nn.Module):
    def __init__(self, num_classes=2, n_mfcc=13, max_time_steps=156):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3))
        self.fc1 = nn.Linear(64 * ((n_mfcc // 4) * (max_time_steps // 4)), 128)  # Fully connected layer
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Conv2D + ReLU
        x = self.pool(x)  # MaxPooling2D
        x = F.relu(self.conv2(x))  # Conv2D + ReLU
        x = self.pool(x)  # MaxPooling2D
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))  # Dense + ReLU
        x = self.dropout(x)  # Dropout
        x = self.fc2(x)  # Output layer
        return x

In [12]:
# 모델 초기화
model = CNNModel(num_classes=NUM_CLASSES, n_mfcc=N_MFCC, max_time_steps=max_time_steps)

# 옵티마이저와 손실 함수 정의
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [13]:
# 모델 학습 함수 정의
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # 기울기 초기화
            outputs = model(inputs)  # 모델 출력
            loss = criterion(outputs, labels)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# 모델 검증 함수 정의
def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct.double() / len(val_loader.dataset)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

In [14]:
# 모델 학습 및 검증
train(model, train_loader, criterion, optimizer, num_epochs=10)
validate(model, val_loader, criterion)

NameError: name 'train_loader' is not defined

In [15]:
# 모델 저장 함수 정의
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [16]:
save_model(model, "audio_classifier.pth")

Model saved to audio_classifier.pth
